In [1]:
# Based on https://gist.github.com/AO8/63b9a5acb9fb238cbed13a0269d14137
# Collects and parses data posted by OC Health
import csv
from urllib.request import urlopen
from bs4 import BeautifulSoup
import datetime
import json

In [2]:
url = "https://occovid19.ochealthinfo.com/coronavirus-in-oc"
out_filename = "oc_data.json"

def get_todays_info(the_url):
    html = urlopen(url)
    soup = BeautifulSoup(html, "html.parser")
    tables = soup.findAll("table")
    tests_table = tables[0]
    city_table = tables[1]
    demo_table = tables[2]
    return (tests_table, city_table, demo_table, soup)

def html_table_to_list(table):
    list_table = []
    for row in table.findAll("tr"):
        row_table = []
        for cell in row.findAll(["td", "th"]):
            row_table.append(cell.get_text())
        list_table.append(row_table)
    return list_table

def get_todays_lists(the_url):
    tables = get_todays_info(the_url)
    the_lists = [html_table_to_list(table) for table in tables]
    todays_date_string = str(tables[-1].text.split('Posted Date:')[-1].split('\n')[0].replace(' ',''))
    return the_lists, todays_date_string

def get_todays_json(the_url):
    # Get lists from today's tables
    todays_lists, todays_date_string = get_todays_lists(the_url)
    
    todays_json = {}
    todays_json["date"] = todays_date_string
    todays_json["tests"] = {}
    todays_json["populations"] = {}
    todays_json["city_cases"] = {}
    for row in todays_lists[1][1:]:
        if row[0] != 'Total Population':
            todays_json["city_cases"][row[0]] = int(row[-1])
    for row in todays_lists[1][1:-5]:
        todays_json["populations"][row[0]] = int(row[1].replace(',', ''))
    todays_json["tests"]["people_tested"] = int(todays_lists[0][0][-1].replace(',', ''))
    todays_json["tests"]["kits_available"] = int(todays_lists[0][1][-1].replace(',', '').replace('specimens',''))

    todays_json["Stats"] = {}
    rows = ["TotalCases", "Deaths", "TravelRelated", "PersonToPerson", "CommunityAcquired", "UnderInvestigation"]
    cols = ["Total", "Male", "Female", "OtherGender", "Under18", "18to49", "50to64", "65andUp", "UnknownAge"]
    for i,row in enumerate(todays_lists[2][3:-1]):
        todays_json["Stats"][rows[i]] = {}
        for j,col in enumerate(cols):
            todays_json["Stats"][rows[i]][cols[j]] = int(row[j+1].replace('\n','').replace(',',''))
    print(todays_date_string)
    return todays_date_string, todays_json

def append_todays_json(the_out_filename, the_url):
    date_string, todays_json = get_todays_json(url)
    with open(the_out_filename, 'r') as saved_json_file:
        saved_json = json.load(saved_json_file)
    if date_string not in saved_json:
        out_json = saved_json
        out_json[date_string] = todays_json
        with open(the_out_filename, 'w') as out_json_file:
            json.dump(out_json, out_json_file)

In [3]:
# First time, just output a base file
# todays_json = get_todays_json(url)
# json_base = {}
# json_base["March 26, 2020"] = todays_json
# with open(out_filename, 'w') as file:
#     json.dump(json_base, file)

In [4]:
# Append to the base file if today's data is different
append_todays_json(out_filename, url)

NameError: name 'date_string' is not defined

In [5]:
todays_info = get_todays_info(url)